In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import os
import re
import numpy as np

### Loading the Data

In [175]:
# File paths
csv1_path = "./Data/MBTA_2024_System-Wide_Passenger_Survey.csv"
parquet_path = "./Data/mbta_arrival_departure_merged_direction.parquet"
csv2_path = "./Data/MBTA_Bus_Ridership_2016-2024.csv"

# Loading
survey_df = pd.read_csv(csv1_path, nrows=30000000)
ridership_df = pd.read_csv(csv2_path, nrows=30000000)

# For Parquet
arrival_departure_df = pd.read_parquet(parquet_path, dtype_backend="pyarrow").head(30000000)

print("survey_df:", survey_df.shape)
print("ridership_df:", ridership_df.shape)
print("arrival_departure_df:", arrival_departure_df.shape)


/var/folders/zs/0gklbd017c758v1gqchfcyk00000gn/T/ipykernel_1775/849930714.py:8: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  ridership_df = pd.read_csv(csv2_path, nrows=30000000)


survey_df: (13866, 8)
ridership_df: (7879638, 17)
arrival_departure_df: (30000000, 14)


In [176]:
display(survey_df.head())

,aggregation_level,service_mode,reporting_group,measure_group,measure,category,weighted_percent,ObjectId
0,Reporting Group,Bus,10,Alternative Modes,Alternative Mode,Drive or Ride in a Carpool,0.316855,1
1,Reporting Group,Bus,10,Alternative Modes,Alternative Mode,"Bike, Scooter, or Other Micromobility",0.000000,2
2,Reporting Group,Bus,10,Alternative Modes,Alternative Mode,Private Shuttle or Other Transit,0.000000,3
3,Reporting Group,Bus,10,Alternative Modes,Alternative Mode,Other,0.000000,4
4,Reporting Group,Bus,10,Alternative Modes,Used Alternative Mode,No,0.837547,5


In [177]:
display(arrival_departure_df.head())

,service_date,route_id,stop_id,half_trip_id,time_point_id,time_point_order,actual,scheduled,scheduled_headway,headway,point_type,standard_type,earliness,direction
0,2018-08-01T00:00:00Z,01,75,40121394,mit,4,1900-01-01T05:19:34Z,1900-01-01T05:19:00Z,<NA>,<NA>,Midpoint,Schedule,-10.0,inbound
1,2018-08-01T00:00:00Z,01,79,40121394,hynes,5,1900-01-01T05:23:20Z,1900-01-01T05:22:00Z,<NA>,<NA>,Midpoint,Schedule,3.0,inbound
2,2018-08-01T00:00:00Z,01,187,40121394,masta,6,1900-01-01T05:25:58Z,1900-01-01T05:25:00Z,<NA>,<NA>,Midpoint,Schedule,-33.0,inbound
3,2018-08-01T00:00:00Z,01,59,40121394,Wasma,7,1900-01-01T05:28:26Z,1900-01-01T05:28:00Z,<NA>,<NA>,Midpoint,Schedule,7.0,inbound
4,2018-08-01T00:00:00Z,01,110,40121565,hhgat,1,1900-01-01T05:29:57Z,1900-01-01T05:30:00Z,1200.0,1218.0,Startpoint,Headway,<NA>,inbound


In [178]:
display(ridership_df.head())

,alightings,boardings,day_type_id,day_type_name,direction_id,load_,route_id,route_variant,sample_size,season,stop_id,stop_name,stop_sequence,trip_start_time,source_file,term_from_name,year_from_name
0,0.3,0.4,day_type_01,weekday,0,7.8,1,1-0-0,13,Fall 2016,1,WASHINGTON ST OPP RUGGLES ST,2,04:37:00,MBTA_Bus_Ridership_by_Trip_Season_Route_Line_a...,Fall,2016
1,0.0,0.1,day_type_01,weekday,0,9.4,1,1-0-0,13,Fall 2016,10003,ALBANY ST OPP RANDALL ST,5,04:37:00,MBTA_Bus_Ridership_by_Trip_Season_Route_Line_a...,Fall,2016
2,0.4,0.0,day_type_01,weekday,0,10.4,1,1-0-0,13,Fall 2016,101,MASSACHUSETTS AVE @ SIDNEY ST,19,04:37:00,MBTA_Bus_Ridership_by_Trip_Season_Route_Line_a...,Fall,2016
3,2.5,0.6,day_type_01,weekday,0,8.5,1,1-0-0,13,Fall 2016,102,MASSACHUSETTS AVE @ PROSPECT,20,04:37:00,MBTA_Bus_Ridership_by_Trip_Season_Route_Line_a...,Fall,2016
4,0.2,0.1,day_type_01,weekday,0,8.4,1,1-0-0,13,Fall 2016,104,MASSACHUSETTS AVE @ BIGELOW S,21,04:37:00,MBTA_Bus_Ridership_by_Trip_Season_Route_Line_a...,Fall,2016


In [179]:
ridership_df['route_id'].unique()[:200]

array([1, 10, 100, 101, 104, 105, 106, 108, 109, 11, 110, 111, 112, 114,
       116, 117, 119, 120, 121, 131, 132, 134, 136, 137, 14, 15, 16, 17,
       170, 171, 18, 19, 201, 202, 21, 210, 211, 212, 214, 215, 216, 217,
       22, 220, 221, 222, 225, 23, 230, 236, 238, 24, 240, 245, 26, 27,
       28, 29, 30, 31, 32, '32', '325', '326', '33', '34', '34E', '35',
       '350', '352', '351', '354', '36', '37', 37, 38, 39, 4, 40, 41, 411,
       42, 424, 426, 428, 429, 43, 430, 434, 435, 436, 439, 44, 441, 442,
       448, 449, 45, 450, 451, 455, 456, 459, 465, 47, 5, 50, 501, 502,
       503, 504, 505, 51, 52, 55, 553, 554, 556, 558, 57, '57', '57A',
       '59', '60', '62', '64', '65', '66', 66, 67, 68, 69, 7, 70, '70',
       '701', '708', '70A', '71', '72', '73', 73, 74, 741, 742, 746, 747,
       749, 75, 751, 76, 77, 78, 79, 8, 80, 83, 84, 85, 86, 87, 88, 89, 9,
       90, 91, 92, 93, 94, 95, 96, 97, 99, '31', 36, 65, '712', '713',
       713, 72, 325, 326, 33, 34, 352, 350, 351, 354

### Data Cleaning

In [180]:
def 3normalize_route(val):
    Normalize route IDs to comparable strings
    if pd.isna(val):
        return None
    s = str(val).strip().upper()
    if s.isdigit():
        s = s.lstrip('0') or '0'
    return s

arrival_departure_df['route_id_norm'] = arrival_departure_df['route_id'].apply(normalize_route)
ridership_df['route_id_norm'] = ridership_df['route_id'].apply(normalize_route)


In [181]:
arrival_departure_df['route_id'] = arrival_departure_df['route_id'].apply(normalize_route)
survey_df['reporting_group'] = survey_df['reporting_group'].apply(normalize_route)
ridership_df['route_id'] = ridership_df['route_id'].apply(normalize_route)

In [182]:
arrival_routes = set(arrival_departure_df['route_id_norm'].dropna().unique())
ridership_routes = set(ridership_df['route_id_norm'].dropna().unique())

common_routes = sorted(arrival_routes & ridership_routes)
only_in_arrival = sorted(arrival_routes - ridership_routes)
only_in_ridership = sorted(ridership_routes - arrival_routes)

print("Routes present in both datasets:")
print(common_routes)

print("\nRoutes only in arrival_departure_df:")
print(only_in_arrival)

print("\n Routes only in ridership_df:")
print(only_in_ridership)


Routes present in both datasets:
['1', '10', '100', '101', '104', '105', '106', '108', '109', '11', '110', '111', '112', '114', '116', '117', '119', '120', '121', '131', '132', '134', '136', '137', '14', '15', '16', '17', '170', '171', '18', '19', '201', '202', '21', '210', '211', '212', '214', '215', '216', '217', '22', '220', '221', '222', '225', '23', '230', '236', '238', '24', '240', '245', '26', '27', '28', '29', '30', '31', '32', '325', '326', '33', '34', '34E', '35', '350', '351', '352', '354', '36', '37', '38', '39', '4', '40', '41', '411', '42', '424', '426', '428', '429', '43', '430', '434', '435', '436', '439', '44', '441', '442', '448', '449', '45', '450', '451', '455', '456', '459', '465', '47', '5', '50', '501', '502', '503', '504', '505', '51', '52', '55', '553', '554', '556', '558', '57', '57A', '59', '60', '62', '64', '65', '66', '67', '68', '69', '7', '70', '70A', '71', '712', '713', '72', '73', '74', '743', '75', '76', '77', '78', '79', '8', '80', '83', '84', '85', '

### Fixing Route Names

In [183]:
arrival_departure_df['route_id'] = arrival_departure_df['route_id'].astype(str).str.strip()
survey_df['reporting_group'] = survey_df['reporting_group'].astype(str).str.strip()
ridership_df['route_id'] = ridership_df['route_id'].astype(str).str.strip()

In [184]:
ridership_df['route_id'] = ridership_df['route_id'].replace('746_', '746')
arrival_departure_df['route_id'] = arrival_departure_df['route_id'].replace('746_', '746')

In [185]:
print("Any '746_' left?", (ridership_df['route_id'] == '746_').any())
print("Any '746' present?", (ridership_df['route_id'] == '746').any())

Any '746_' left? False
Any '746' present? True


In [186]:
print("Any '746_' left?", (arrival_departure_df['route_id'] == '746_').any())
print("Any '746' present?", (arrival_departure_df['route_id'] == '746').any())

Any '746_' left? False
Any '746' present? True


### Identifying Common Bus Routes

In [187]:
# Unique sets
arrival_routes = set(arrival_departure_df['route_id'].dropna().unique())
survey_routes = set(survey_df['reporting_group'].dropna().unique())

# Compare
common_routes_1 = sorted(arrival_routes & survey_routes)
only_in_arrival_1 = sorted(arrival_routes - survey_routes)
only_in_survey_1 = sorted(survey_routes - arrival_routes)

print("Routes present in both (arrival_departure_df & survey_df):")
print(common_routes_1)

print("\n Routes only in arrival_departure_df:")
print(only_in_arrival_1)

print("\n Routes only in survey_df:")
print(only_in_survey_1)

Routes present in both (arrival_departure_df & survey_df):
['1', '10', '104', '11', '111', '15', '16', '21', '22', '23', '32', '39', '47', '57', '66', '71', '73', '86', '9', 'SL1', 'SL2', 'SL4']

 Routes only in arrival_departure_df:
['100', '101', '105', '106', '108', '109', '110', '112', '114', '116', '117', '119', '120', '121', '131', '132', '134', '136', '137', '14', '17', '170', '171', '18', '19', '191', '192', '193', '194', '195', '201', '202', '210', '211', '212', '214', '215', '216', '217', '220', '221', '222', '225', '230', '236', '238', '24', '240', '245', '26', '27', '28', '29', '30', '31', '325', '326', '33', '34', '34E', '35', '350', '351', '352', '354', '36', '37', '38', '4', '40', '41', '411', '42', '424', '426', '428', '429', '43', '430', '434', '435', '436', '439', '44', '441', '442', '448', '449', '45', '450', '451', '455', '456', '459', '465', '5', '50', '501', '502', '503', '504', '505', '51', '52', '55', '553', '554', '556', '558', '57A', '59', '60', '62', '6276', 

In [188]:
# Unique sets
ridership_routes = set(ridership_df['route_id'].dropna().unique())
survey_routes = set(survey_df['reporting_group'].dropna().unique())

# Compare
common_routes_2 = sorted(ridership_routes & survey_routes)
only_in_ridership_2 = sorted(ridership_routes - survey_routes)
only_in_survey_2 = sorted(survey_routes - ridership_routes)

print("Routes present in both (ridership_df & survey_df):")
print(common_routes_2)

print("\n Routes only in ridership_df:")
print(only_in_ridership_2)

print("\n Routes only in survey_df:")
print(only_in_survey_2)

Routes present in both (ridership_df & survey_df):
['1', '10', '104', '11', '111', '15', '16', '21', '22', '23', '32', '39', '47', '57', '66', '71', '73', '86', '9']

 Routes only in ridership_df:
['100', '101', '105', '106', '108', '109', '110', '112', '114', '116', '117', '119', '120', '121', '131', '132', '134', '136', '137', '14', '17', '170', '171', '18', '19', '201', '202', '210', '211', '212', '214', '215', '216', '217', '220', '221', '222', '225', '226', '230', '236', '238', '24', '240', '245', '26', '27', '28', '29', '30', '31', '325', '326', '33', '34', '34E', '35', '350', '351', '352', '354', '36', '37', '38', '4', '40', '41', '411', '42', '424', '426', '428', '429', '43', '430', '434', '435', '436', '439', '44', '441', '442', '448', '449', '45', '450', '451', '455', '456', '459', '465', '5', '50', '501', '502', '503', '504', '505', '51', '52', '55', '553', '554', '556', '558', '57A', '59', '60', '61', '62', '64', '65', '67', '68', '69', '7', '70', '701', '708', '70A', '712'

#### We see that some columns in survey_df are named in the form route_id_1& route_id_2. We adress that issue here.

In [189]:
ROUTE_TOKEN_RE = re.compile(r'\b(?:SL\d+|CT\d+|\d+[A-Z]?)\b', flags=re.IGNORECASE)

def normalize_route(s):
    if pd.isna(s): return None
    s = str(s).strip().upper()
    s = re.sub(r'[^A-Z0-9]+', '', s)
    if not s: return None
    if s.isdigit():
        s = s.lstrip('0') or '0'
    return s

def extract_route_tokens(text):
    """Return list of normalized routes from a reporting_group string."""
    if pd.isna(text):
        return []
    s = str(text).upper()
    tokens = ROUTE_TOKEN_RE.findall(s)
    out = []
    for t in tokens:
        out.append(t.lstrip('0') if t.isdigit() else t)
    # dedupe preserving order
    seen, uniq = set(), []
    for t in out:
        if t not in seen:
            seen.add(t)
            uniq.append(t)
    return uniq

# Adding the new column
survey_df['route_id'] = survey_df['reporting_group'].map(extract_route_tokens)

print(survey_df[['reporting_group', 'route_id']].head(10))


  reporting_group route_id
0              10     [10]
1              10     [10]
2              10     [10]
3              10     [10]
4              10     [10]
5              10     [10]
6              10     [10]
7              10     [10]
8              10     [10]
9              10     [10]


#### Now we check route_ids again with our updated dataframe

In [190]:
# ARRIVAL vs SURVEY 

arrival_routes = set(arrival_departure_df['route_id'].dropna().astype(str).unique())

survey_routes = set(
    r
    for lst in survey_df['route_id'].dropna()
    for r in (lst if isinstance(lst, list) else [])
)

# Compare
common_routes_1 = sorted(arrival_routes & survey_routes)
only_in_arrival_1 = sorted(arrival_routes - survey_routes)
only_in_survey_1 = sorted(survey_routes - arrival_routes)

print("Routes present in both (arrival_departure_df & survey_df):")
print(common_routes_1)

print("\nRoutes only in arrival_departure_df:")
print(only_in_arrival_1)

print("\nRoutes only in survey_df:")
print(only_in_survey_1)

Routes present in both (arrival_departure_df & survey_df):
['1', '10', '104', '11', '111', '114', '116', '117', '15', '16', '21', '22', '23', '28', '29', '32', '34', '34E', '39', '4', '43', '47', '55', '57', '66', '67', '7', '70', '71', '73', '77', '8', '86', '9', 'CT3', 'SL1', 'SL2', 'SL4', 'SL5']

Routes only in arrival_departure_df:
['100', '101', '105', '106', '108', '109', '110', '112', '119', '120', '121', '131', '132', '134', '136', '137', '14', '17', '170', '171', '18', '19', '191', '192', '193', '194', '195', '201', '202', '210', '211', '212', '214', '215', '216', '217', '220', '221', '222', '225', '230', '236', '238', '24', '240', '245', '26', '27', '30', '31', '325', '326', '33', '35', '350', '351', '352', '354', '36', '37', '38', '40', '41', '411', '42', '424', '426', '428', '429', '430', '434', '435', '436', '439', '44', '441', '442', '448', '449', '45', '450', '451', '455', '456', '459', '465', '5', '50', '501', '502', '503', '504', '505', '51', '52', '553', '554', '556',

In [191]:
# RIDERSHIP vs SURVEY 

ridership_routes = set(ridership_df['route_id'].dropna().astype(str).unique())

survey_routes = set(
    r
    for lst in survey_df['route_id'].dropna()
    for r in (lst if isinstance(lst, list) else [])
)

# Compare
common_routes_2 = sorted(ridership_routes & survey_routes)
only_in_ridership_2 = sorted(ridership_routes - survey_routes)
only_in_survey_2 = sorted(survey_routes - ridership_routes)

print("Routes present in both (ridership_df & survey_df):")
print(common_routes_2)

print("\nRoutes only in ridership_df:")
print(only_in_ridership_2)

print("\nRoutes only in survey_df:")
print(only_in_survey_2)


Routes present in both (ridership_df & survey_df):
['1', '10', '104', '11', '111', '114', '116', '117', '15', '16', '21', '22', '23', '28', '29', '32', '34', '34E', '39', '4', '43', '47', '55', '57', '61', '66', '67', '7', '70', '71', '73', '77', '8', '86', '9']

Routes only in ridership_df:
['100', '101', '105', '106', '108', '109', '110', '112', '119', '120', '121', '131', '132', '134', '136', '137', '14', '17', '170', '171', '18', '19', '201', '202', '210', '211', '212', '214', '215', '216', '217', '220', '221', '222', '225', '226', '230', '236', '238', '24', '240', '245', '26', '27', '30', '31', '325', '326', '33', '35', '350', '351', '352', '354', '36', '37', '38', '40', '41', '411', '42', '424', '426', '428', '429', '430', '434', '435', '436', '439', '44', '441', '442', '448', '449', '45', '450', '451', '455', '456', '459', '465', '5', '50', '501', '502', '503', '504', '505', '51', '52', '553', '554', '556', '558', '57A', '59', '60', '62', '64', '65', '68', '69', '701', '708', '7

In [192]:
unique_survey_routes = sorted(
    {r for lst in survey_df['route_id'].dropna() for r in lst if isinstance(lst, list)}
)

print("Unique route_ids in survey_df:")
print(unique_survey_routes)
print("\nTotal unique routes:", len(unique_survey_routes))


Unique route_ids in survey_df:
['1', '10', '104', '11', '111', '114', '116', '117', '15', '16', '21', '22', '23', '28', '29', '32', '34', '34E', '39', '4', '43', '47', '55', '57', '61', '66', '67', '7', '70', '71', '73', '77', '8', '86', '9', 'CT3', 'SL1', 'SL2', 'SL3', 'SL4', 'SL5']

Total unique routes: 41


In [193]:
# Show all unique reporting_group entries
unique_reporting_groups = survey_df['reporting_group'].dropna().unique()

print("Unique reporting_group values in survey_df:")
for val in unique_reporting_groups:
    print(val)

print("\nTotal unique reporting_groups:", len(unique_reporting_groups))


Unique reporting_group values in survey_df:
10
1
104
111
11
21
114, 116, & 117
15
16
22
28 & 29
23
32
34 & 34E
39
43 & SL5
4 & 7
47
57
66
61 & 70
67 & 77
71
73
9
86
BRIGHTON/ALLSTON/WATERTOWN - BOSTON
8, 55 & CT3
BRIGHTON/ALLSTON/WATERTOWN - OTHER SUBURB
CHELSEA/EAST BOSTON/LYNN/REVERE - CORE
CAMBRIDGE/ARLINGTON - ALEWIFE
CAMBRIDGE/ARLINGTON - HARVARD
CHELSEA/EAST BOSTON/LYNN/REVERE - NORTH SHORE
MALDEN/EVERETT - SOUTH
MALDEN/EVERETT - NORTH
QUINCY - EAST
JAMAICA PLAIN/ROSLINDALE/HYDE PARK
ROXBURY/DORCHESTER/MATTAPAN/SOUTH END - EAST
QUINCY - WEST
ROXBURY/DORCHESTER/MATTAPAN/SOUTH END & JAMAICA PLAIN/ROSLINDALE/HYDE PARK
ROXBURY/DORCHESTER/MATTAPAN/SOUTH END - WEST
SL4
SOMERVILLE/MEDFORD/CHARLESTOWN - EAST
FITCHBURG LINE
SOMERVILLE/MEDFORD/CHARLESTOWN - WEST
FAIRMOUNT LINE
FRAMINGHAM/WORCESTER LINE
GREENBUSH LINE
FRANKLIN/FOXBORO LINE
KINGSTON LINE
HAVERHILL LINE
NEEDHAM LINE
NEWBURYPORT/ROCKPORT LINE
MIDDLEBOROUGH/LAKEVILLE LINE
LOWELL LINE
PROVIDENCE/STOUGHTON LINE
FERRY
AIRPORT
AQUA

#### Dropping routes with invalid bus_ids

In [194]:
# Drop survey rows that have no route tokens
survey_df = survey_df[survey_df['route_id'].map(len) > 0].reset_index(drop=True)

print("Cleaned survey_df — only rows with valid route IDs remain.")
print("Remaining rows:", len(survey_df))
print("Unique reporting_groups after cleaning:", survey_df['reporting_group'].nunique())


Cleaned survey_df — only rows with valid route IDs remain.
Remaining rows: 2685
Unique reporting_groups after cleaning: 31


### Only keeping rows with route_id present in all dataframes

In [195]:
import os
import pandas as pd

def normalize_route(val):
    #Normalize route IDs to comparable strings
    if pd.isna(val):
        return None
    s = str(val).strip().upper()
    if s.isdigit():
        s = s.lstrip('0') or '0'
    return s

def norm_list(list_val):
    #Normalize a list of route_ids (possibly None/NaN)
    if not isinstance(list_val, list):
        return []
    out = []
    for v in list_val:
        nv = normalize_route(v)
        if nv is not None:
            out.append(nv)
    return out

#  Normalize columns
ridership_df = ridership_df.assign(
    route_id_norm=ridership_df['route_id'].apply(normalize_route)
)

arrival_departure_df = arrival_departure_df.assign(
    route_id_norm=arrival_departure_df['route_id'].apply(normalize_route)
)

# For survey, the route_id column holds lists; make a normalized companion list column
# and also build the unique set from those lists
survey_df = survey_df.assign(
    route_id_list_norm=survey_df['route_id'].apply(norm_list)
)

unique_survey_routes = set()
for lst in survey_df['route_id_list_norm']:
    unique_survey_routes.update(lst)

# Build intersections 
ridership_routes = set(r for r in ridership_df['route_id_norm'].dropna().unique())
arrival_routes   = set(r for r in arrival_departure_df['route_id_norm'].dropna().unique())
survey_routes    = unique_survey_routes

common_routes = ridership_routes & arrival_routes & survey_routes

print(f"# of routes -> ridership: {len(ridership_routes)}, arrival: {len(arrival_routes)}, survey: {len(survey_routes)}")
print(f"# of common routes across all three: {len(common_routes)}")

# of routes -> ridership: 173, arrival: 180, survey: 41
# of common routes across all three: 34


In [197]:
import os
import numpy as np
import pandas as pd

#  caps
ARRIVAL_MAX   = 30000000
RIDERSHIP_MAX = 30000000
SURVEY_MAX    = 30000000  

# NumPy-only slicer
def numpy_slice_frame(df: pd.DataFrame, idx: np.ndarray) -> pd.DataFrame:
    out = {}
    for c in df.columns:
        col_np = df[c].to_numpy()
        out[c] = np.take(col_np, idx, axis=0)
    return pd.DataFrame(out)

# build indices of matching rows 
arr_route_np = arrival_departure_df["route_id_norm"].astype("string[python]").to_numpy()
rid_route_np = ridership_df["route_id_norm"].astype("string[python]").to_numpy()

mask_arr = np.fromiter((r in common_routes for r in arr_route_np), count=arr_route_np.size, dtype=bool)
mask_rid = np.fromiter((r in common_routes for r in rid_route_np), count=rid_route_np.size, dtype=bool)

idx_arr_all = np.flatnonzero(mask_arr)
idx_rid_all = np.flatnonzero(mask_rid)

# apply caps before slicing 
idx_arr = idx_arr_all[:ARRIVAL_MAX]
idx_rid = idx_rid_all[:RIDERSHIP_MAX]

# survey: keep row if ANY route in its list is in common_routes
svy_lists = survey_df["route_id_list_norm"].to_numpy(object)
mask_svy = np.fromiter((bool(L) and any(r in common_routes for r in L) for L in svy_lists),
                       count=svy_lists.size, dtype=bool)
idx_svy_all = np.flatnonzero(mask_svy)
idx_svy = idx_svy_all[:SURVEY_MAX]

print(f"Will materialize -> arrival: {idx_arr.size:,}; ridership: {idx_rid.size:,}; survey: {idx_svy.size:,}")

arrival_df   = numpy_slice_frame(arrival_departure_df, idx_arr)
ridership_ok = numpy_slice_frame(ridership_df, idx_rid)
survey_ok    = numpy_slice_frame(survey_df, idx_svy)

for _df, cols in [
    (arrival_df,   ["route_id_norm"]),
    (ridership_ok, ["route_id_norm"]),
    (survey_ok,    ["route_id_list_norm"]),
]:
    drop_cols = [c for c in cols if c in _df.columns]
    if drop_cols:
        _df.drop(columns=drop_cols, inplace=True, errors="ignore")

# save 
os.makedirs("./data_cleaned_capped", exist_ok=True)

arrival_df.to_parquet("./data_cleaned_capped/arrival_departure.parquet")

def write_csv_in_chunks(df: pd.DataFrame, path: str, chunksize: int = 50_000):
    first = True
    n = len(df)
    for start in range(0, n, chunksize):
        end = min(start + chunksize, n)
        block_idx = np.arange(start, end, dtype=np.int64)
        block = numpy_slice_frame(df, block_idx)
        block.to_csv(path, index=False, mode=("w" if first else "a"), header=first)
        first = False

write_csv_in_chunks(ridership_ok, "./data_cleaned_capped/ridership.csv", chunksize=10_000)
write_csv_in_chunks(survey_ok,    "./data_cleaned_capped/survey.csv",    chunksize=10_000)

print("Saved capped outputs to ./data_cleaned/: arrival_departure.parquet, ridership.csv, survey.csv")


Will materialize -> arrival: 12,682,058; ridership: 3,327,451; survey: 2,335
Saved capped outputs to ./data_cleaned/: arrival_departure.parquet, ridership.csv, survey.csv
